In [1]:
# importing libraries

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 

from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler

import torch 
from torch import nn 

In [2]:
# Define the batch size and model save paths

BATCH_SIZE = 128
PATH = 'Models/02_Sequential_10M/'

In [3]:
# Loading the dataset

data = pd.read_csv('ik_dataset_2.csv')
data.head()

,joint_0,joint_1,joint_2,joint_3,joint_4,joint_5,x,y,z
0,-1.490554,-0.575344,-2.497704,0.783136,-0.794136,-5.090668,-0.003073,-0.369845,0.974694
1,1.268094,-0.673851,-1.588101,-3.159301,1.822181,5.669746,0.175270,0.564963,0.857058
2,1.139133,0.082702,0.816177,-1.139029,-0.535171,6.465722,0.012923,0.100015,0.040055
3,0.625910,0.619670,-0.711847,-0.362054,-1.497531,-5.998428,0.358412,0.287417,-0.234620
4,2.481490,0.952712,-2.911307,0.137935,0.263415,-5.043665,-0.550294,0.424230,0.210514


In [4]:
# Normalising the data 

scaler = MinMaxScaler()
data = pd.DataFrame(scaler.fit_transform(data))
data.columns = ['joint_0','joint_1','joint_2','joint_3','joint_4','joint_5','x','y','z']
data.head()

,joint_0,joint_1,joint_2,joint_3,joint_4,joint_5,x,y,z
0,0.262770,0.335176,0.274615,0.612176,0.310414,0.135407,0.498048,0.288605,0.886813
1,0.701823,0.306955,0.460745,0.047463,0.935015,0.906066,0.600031,0.823207,0.813741
2,0.681298,0.523692,0.952729,0.336846,0.372238,0.963074,0.507195,0.557311,0.306247
3,0.599616,0.677522,0.640052,0.448139,0.142491,0.070393,0.704759,0.664483,0.135628
4,0.894941,0.772932,0.189980,0.519757,0.562886,0.138773,0.185126,0.742725,0.412130


In [5]:
# Device agnostic code

if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
device

'cuda'

In [6]:
# Converting data to tensors 

xtest = torch.Tensor(data[['x','y','z']].to_numpy())
ytest = torch.Tensor(data[['joint_0','joint_1','joint_2','joint_3','joint_4','joint_5']].to_numpy())

xtest.shape, ytest.shape

(torch.Size([1000000, 3]), torch.Size([1000000, 6]))

In [7]:

# Defining the model 

class InverseKinematicsABBIRB140(nn.Module):
    
    """ This is a model structure specifically designed to calculate the inverse kinematics of the industrial robot ABB IRB 140, but can be used for any robot
        This has a layer structure of input -> 64 -> 128 -> 64 -> output with non-linear activation and normalisation layers
    Args:
        input_features (int): defines the number of features given as input to the model
        output_features (int): defines the number of features the model outputs
    """
    
    def __init__(self, input_features, output_features):
        super().__init__()
        self.layer_stack_1 = nn.Sequential(
            nn.Linear(in_features=input_features, out_features=128),
            nn.ReLU(),
            nn.BatchNorm1d(num_features=128),
            nn.Dropout()
        )
        self.layer_stack_2 = nn.Sequential(
            nn.Linear(in_features=128, out_features=128),
            nn.ReLU(),
            nn.BatchNorm1d(num_features=128),
            nn.Dropout()
        )
        self.layer_stack_3 = nn.Sequential(
            nn.Linear(in_features=128, out_features=128),
            nn.ReLU(),
            nn.BatchNorm1d(num_features=128),
            nn.Dropout()
        )
        self.layer_stack_4 = nn.Sequential(
            nn.Linear(in_features=128, out_features=128),
            nn.ReLU(),
            nn.BatchNorm1d(num_features=128),
            nn.Dropout(),
            nn.Linear(in_features=128, out_features=output_features)
        )
    
    def forward(self, x):
        x = self.layer_stack_1(x)
        x = self.layer_stack_2(x)
        x = self.layer_stack_3(x)
        x = self.layer_stack_4(x)
        return x   
    

In [8]:
# Instantiating various models used 

model_0 = InverseKinematicsABBIRB140(input_features=3,output_features=1)
model_1 = InverseKinematicsABBIRB140(input_features=4,output_features=1)
model_2 = InverseKinematicsABBIRB140(input_features=5,output_features=1)
model_3 = InverseKinematicsABBIRB140(input_features=6,output_features=1)
model_4 = InverseKinematicsABBIRB140(input_features=7,output_features=1)
model_5 = InverseKinematicsABBIRB140(input_features=8,output_features=1)

model_0.load_state_dict(torch.load(f=PATH+"model_0.pth"))
model_1.load_state_dict(torch.load(f=PATH+"model_1.pth"))
model_2.load_state_dict(torch.load(f=PATH+"model_2.pth"))
model_3.load_state_dict(torch.load(f=PATH+"model_3.pth"))
model_4.load_state_dict(torch.load(f=PATH+"model_4.pth"))
model_5.load_state_dict(torch.load(f=PATH+"model_5.pth"))

<All keys matched successfully>

In [9]:
# Calculate all the joint angles from our models

model_0.eval()
with torch.inference_mode():
    joint_0_pred = model_0(xtest)
    
model_1.eval()
with torch.inference_mode():
    joint_1_pred = model_1(torch.cat((xtest, joint_0_pred), dim=1))
    
model_2.eval()
with torch.inference_mode():
    joint_2_pred = model_2(torch.cat((xtest, joint_0_pred, joint_1_pred), dim=1))
    
model_3.eval()
with torch.inference_mode():
    joint_3_pred = model_3(torch.cat((xtest, joint_0_pred, joint_1_pred, joint_2_pred), dim=1))
    
model_4.eval()
with torch.inference_mode():
    joint_4_pred = model_4(torch.cat((xtest, joint_0_pred, joint_1_pred, joint_2_pred, joint_3_pred), dim=1))
    
model_5.eval()
with torch.inference_mode():
    joint_5_pred = model_5(torch.cat((xtest, joint_0_pred, joint_1_pred, joint_2_pred, joint_3_pred, joint_4_pred), dim=1))

ypred = torch.cat((joint_0_pred, joint_1_pred, joint_2_pred, joint_3_pred, joint_4_pred, joint_5_pred), dim=1)

In [10]:
ypred[:5], ytest[:5]

(tensor([[0.3512, 0.2365, 0.3941, 0.5058, 0.4158, 0.5105],
         [0.6862, 0.2931, 0.4594, 0.5059, 0.5642, 0.5116],
         [0.4281, 0.7174, 0.8999, 0.5049, 0.6645, 0.5108],
         [0.5896, 0.7627, 0.4699, 0.5077, 0.6306, 0.5121],
         [0.8876, 0.5675, 0.4553, 0.5046, 0.5657, 0.5120]]),
 tensor([[0.2628, 0.3352, 0.2746, 0.6122, 0.3104, 0.1354],
         [0.7018, 0.3070, 0.4607, 0.0475, 0.9350, 0.9061],
         [0.6813, 0.5237, 0.9527, 0.3368, 0.3722, 0.9631],
         [0.5996, 0.6775, 0.6401, 0.4481, 0.1425, 0.0704],
         [0.8949, 0.7729, 0.1900, 0.5198, 0.5629, 0.1388]]))

In [11]:
loss_fn_1 = nn.MSELoss()

In [19]:
loss_fn_1(ypred, ytest)

tensor(0.0620)

In [13]:
loss_fn_2 = nn.L1Loss()

In [17]:
loss_fn_2(ypred, ytest)

tensor(0.1975)

### Final Loss

1) Direct Approach :
    1) MSE Loss : 0.1127
    2) L1 Loss :0.2747
2) Sequential Approach :
    1) MSE Loss : 0.0620
    2) L1 Loss : 0.1975